In [4]:
import pandas as pd 
from lead_data_pipeline import get_primary_email, clean_text, clean_phone, clean_website, normalize_province

df = pd.read_csv("/Users/isaie/Lyyvora-outreach-core-service/datasets/real_set_v1/records.csv")

# Step 0, load dirty data
df

,google_place_url,business_name,business_website,business_phone,email_1,email_2,type,sub_types,full_address,borough,...,contact_page,facebook,twitter,instagram,youtube,linkedin,website_built_with,website_title,website_desc,source_file
0,https://www.google.com/maps?cid=11474720609362...,DDG Med Spa (Drop Dead Gorgeous),http://www.ddgmedspa.com/,+1 905-379-4344,info@ddgmedspa.com,NaN,Skin care clinic,"Skin care clinic, Medical spa","11 Hess St S, Hamilton, ON L8P 3M7, Canada",Central Hamilton,...,https://ddgmedspa.com/contact/,https://www.facebook.com/DropDeadGorgeousMedSpa,NaN,https://www.instagram.com/ddgmedspa/,NaN,NaN,Site Kit by Google 1.157.0,DDG Med Spa – Hamilton,NaN,Canada.csv
1,https://www.google.com/maps?cid=14331759844270...,SKYN Med Spa,http://www.skynmedspa.ca/,+1 905-387-7596,NaN,NaN,Skin care clinic,"Skin care clinic, Cosmetic surgeon, Medical spa","576 Concession St Unit B, Hamilton, ON L8V 1B1...",Eastmount,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada.csv
2,https://www.google.com/maps?cid=15825456821524...,Skin Vitality Medical Clinic Hamilton,https://www.skinvitality.ca/hamilton/?src=7014...,+1 365-795-0096,NaN,NaN,Skin care clinic,"Skin care clinic, Beauty product supplier, Fac...","25 Main St W Suite 101, Hamilton, ON L8P 1H1, ...",Durand,...,https://www.skinvitality.ca/contact-us/,https://www.facebook.com/SkinVitalityMedicalCl...,NaN,https://www.instagram.com/skinvitality/,https://youtube.com/shorts/s72OO5SGn6c?si=SoU8...,NaN,WordPress 6.7.2,Botox Hamilton | Lip Fillers Hamilton Near Me ...,Huge Savings on CoolSculpting! - Top CoolSculp...,Canada.csv
3,https://www.google.com/maps?cid=85682291802790...,Gina's Sugar Shack - Organic Skin Care & Hair ...,http://www.ginassugarshack.ca/,+1 613-882-4435,NaN,NaN,Skin care clinic,"Skin care clinic, Boutique, Hair removal service","5548 Manotick Main St, Manotick, ON K4M 1A5, C...",Manotick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada.csv
4,https://www.google.com/maps?cid=86256234926554...,Ethos Rejuvenation Centre,http://www.ethosskincare.com/,+1 519-601-8881,info@ethosskincare.com,NaN,Skin care clinic,"Skin care clinic, Day spa, Facial spa, Laser h...","1140 Southdale Rd W, London, ON N6P 0E1, Canada",Byron,...,https://www.ethosskincare.com/contact-us/,https://www.facebook.com/EthosRejuvenationCentre/,NaN,https://www.instagram.com/ethos_skincare/,NaN,NaN,WordPress 6.8.2,"Medical Rejuvenation Centre in London, Ontario...",Experience the best of medical rejuvenation at...,Canada.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,https://www.google.com/maps?cid=19581920908898...,York Medical Group Inc,https://ymgclinic.com/,+1 647-809-5365,frontdesk@ymgclinic.com,info@ymgclinic.com,Skin care clinic,Skin care clinic,"10014 Yonge St, Richmond Hill, ON L4C 1T8, Canada",NaN,...,https://ymgclinic.com/contact-us/,NaN,NaN,https://www.instagram.com/ymgaestheticclinic/,NaN,NaN,WordPress 6.8.1,Home - Aesthetic Clinic,NaN,Canada.csv
1376,https://www.google.com/maps?cid=76174564134726...,Glow & Go Beauty Medical Spa,http://www.glowngobeauty.ca/,+1 519-817-0700,sandra.glowngobeauty@gmail.com,NaN,Skin care clinic,"Skin care clinic, Cosmetics store, Facial spa,...","7610 Tecumseh Rd E Unit 3, Windsor, ON N8T 1E9...",Forest Glade,...,NaN,http://www.facebook.com/glowngobeauties,NaN,https://instagram.com/glowngobeauties,NaN,NaN,Wix.com Website Builder,Glow & Go Beauty Medical Spa | RN | Botox Inje...,Glow & Go Beauty is certified in advanced cosm...,Canada.csv
1377,https://www.google.com/maps?cid=56704850563301...,Essence Beauty By Joelle,NaN,+1 519-991-6848,NaN,NaN,Skin care clinic,Skin care clinic,"LaSalle, ON N9H 0L5, Canada",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Canada.csv
1378,https://www.google.com/maps?cid=13202086554562...,Klodas Medical Aesthetics,http://www.klodasmedicalaesthetics.ca/,+1 416-535-3229,NaN,NaN,Skin care clinic,"Skin care clinic, Facial spa, Medical spa","2333 Dundas St W Suite 302, Toronto, ON M6R 3A...",O

In [5]:
# Step 1, adjust csv column names to map to sqlite database columns
df = df.rename(columns={
  "business_name": "clinic_name",
  "type": "clinic_main_type",
  "sub_types": "clinic_sub_type",
  "business_website": "website_url",
  "state": "province",
  "business_phone": "phone"
})

# Step 1.1, set email column to first valid email
df["email"] = df.apply(lambda row: get_primary_email(row.get("email_1"), row.get("email_2")), axis=1)

# Step 1.2, remove unnecessary columns, display the 11 relevant columns
df = df[[
        "clinic_name", "clinic_main_type", "clinic_sub_type",
        "city", "province", "phone", "email",
        "website_url", "website_desc", "total_reviews", "average_rating"
]]

df

,clinic_name,clinic_main_type,clinic_sub_type,city,province,phone,email,website_url,website_desc,total_reviews,average_rating
0,DDG Med Spa (Drop Dead Gorgeous),Skin care clinic,"Skin care clinic, Medical spa",Hamilton,Ontario,+1 905-379-4344,info@ddgmedspa.com,http://www.ddgmedspa.com/,NaN,203.0,4.7
1,SKYN Med Spa,Skin care clinic,"Skin care clinic, Cosmetic surgeon, Medical spa",Hamilton,Ontario,+1 905-387-7596,None,http://www.skynmedspa.ca/,NaN,161.0,4.9
2,Skin Vitality Medical Clinic Hamilton,Skin care clinic,"Skin care clinic, Beauty product supplier, Fac...",Hamilton,Ontario,+1 365-795-0096,None,https://www.skinvitality.ca/hamilton/?src=7014...,Huge Savings on CoolSculpting! - Top CoolSculp...,457.0,4.8
3,Gina's Sugar Shack - Organic Skin Care & Hair ...,Skin care clinic,"Skin care clinic, Boutique, Hair removal service",Manotick,Ontario,+1 613-882-4435,None,http://www.ginassugarshack.ca/,NaN,22.0,5.0
4,Ethos Rejuvenation Centre,Skin care clinic,"Skin care clinic, Day spa, Facial spa, Laser h...",London,Ontario,+1 519-601-8881,info@ethosskincare.com,http://www.ethosskincare.com/,Experience the best of medical rejuvenation at...,91.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...
1375,York Medical Group Inc,Skin care clinic,Skin care clinic,Richmond Hill,Ontario,+1 647-809-5365,frontdesk@ymgclinic.com,https://ymgclinic.com/,NaN,19.0,5.0
1376,Glow & Go Beauty Medical Spa,Skin care clinic,"Skin care clinic, Cosmetics store, Facial spa,...",Windsor,Ontario,+1 519-817-0700,sandra.glowngobeauty@gmail.com,http://www.glowngobeauty.ca/,Glow & Go Beauty is certified in advanced cosm...,10.0,5.0
1377,Essence Beauty By Joelle,Skin care clinic,Skin care clinic,LaSalle,Ontario,+1 519-991-6848,None,NaN,NaN,2.0,5.0
1378,Klodas Medical Aesthetics,Skin care clinic,"Skin care clinic, Facial spa, Medical spa",Toronto,Ontario,+1 416-535-3229,None,http://www.klodasmedicalaesthetics.ca/,NaN,5.0,5.0


In [6]:
# Step 2, clean and format text for clinic_name, clinic_main_type, clinic_sub_type, city columns
for col in ["clinic_name", "clinic_main_type", "clinic_sub_type", "city"]:
  df[col] = df[col].apply(clean_text)
  
df

,clinic_name,clinic_main_type,clinic_sub_type,city,province,phone,email,website_url,website_desc,total_reviews,average_rating
0,DDG Med Spa (Drop Dead Gorgeous),Skin care clinic,"Skin care clinic, Medical spa",Hamilton,Ontario,+1 905-379-4344,info@ddgmedspa.com,http://www.ddgmedspa.com/,NaN,203.0,4.7
1,SKYN Med Spa,Skin care clinic,"Skin care clinic, Cosmetic surgeon, Medical spa",Hamilton,Ontario,+1 905-387-7596,None,http://www.skynmedspa.ca/,NaN,161.0,4.9
2,Skin Vitality Medical Clinic Hamilton,Skin care clinic,"Skin care clinic, Beauty product supplier, Fac...",Hamilton,Ontario,+1 365-795-0096,None,https://www.skinvitality.ca/hamilton/?src=7014...,Huge Savings on CoolSculpting! - Top CoolSculp...,457.0,4.8
3,Gina's Sugar Shack - Organic Skin Care & Hair ...,Skin care clinic,"Skin care clinic, Boutique, Hair removal service",Manotick,Ontario,+1 613-882-4435,None,http://www.ginassugarshack.ca/,NaN,22.0,5.0
4,Ethos Rejuvenation Centre,Skin care clinic,"Skin care clinic, Day spa, Facial spa, Laser h...",London,Ontario,+1 519-601-8881,info@ethosskincare.com,http://www.ethosskincare.com/,Experience the best of medical rejuvenation at...,91.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...
1375,York Medical Group Inc,Skin care clinic,Skin care clinic,Richmond Hill,Ontario,+1 647-809-5365,frontdesk@ymgclinic.com,https://ymgclinic.com/,NaN,19.0,5.0
1376,Glow & Go Beauty Medical Spa,Skin care clinic,"Skin care clinic, Cosmetics store, Facial spa,...",Windsor,Ontario,+1 519-817-0700,sandra.glowngobeauty@gmail.com,http://www.glowngobeauty.ca/,Glow & Go Beauty is certified in advanced cosm...,10.0,5.0
1377,Essence Beauty By Joelle,Skin care clinic,Skin care clinic,LaSalle,Ontario,+1 519-991-6848,None,NaN,NaN,2.0,5.0
1378,Klodas Medical Aesthetics,Skin care clinic,"Skin care clinic, Facial spa, Medical spa",Toronto,Ontario,+1 416-535-3229,None,http://www.klodasmedicalaesthetics.ca/,NaN,5.0,5.0


In [7]:
# Step 3, clean and standardize the province column - convert provinces to abbreviations
df["province"] = df["province"].apply(normalize_province)

# Step 3.1, clean and standardize phone column
df["phone"] = df["phone"].apply(clean_phone)
    
# Step 3.2, clean and verify valid website urls
df["website_url"] = df["website_url"].apply(clean_website)

df

,clinic_name,clinic_main_type,clinic_sub_type,city,province,phone,email,website_url,website_desc,total_reviews,average_rating
0,DDG Med Spa (Drop Dead Gorgeous),Skin care clinic,"Skin care clinic, Medical spa",Hamilton,ON,9053794344,info@ddgmedspa.com,http://www.ddgmedspa.com/,NaN,203.0,4.7
1,SKYN Med Spa,Skin care clinic,"Skin care clinic, Cosmetic surgeon, Medical spa",Hamilton,ON,9053877596,None,http://www.skynmedspa.ca/,NaN,161.0,4.9
2,Skin Vitality Medical Clinic Hamilton,Skin care clinic,"Skin care clinic, Beauty product supplier, Fac...",Hamilton,ON,3657950096,None,https://www.skinvitality.ca/hamilton/?src=7014...,Huge Savings on CoolSculpting! - Top CoolSculp...,457.0,4.8
3,Gina's Sugar Shack - Organic Skin Care & Hair ...,Skin care clinic,"Skin care clinic, Boutique, Hair removal service",Manotick,ON,6138824435,None,http://www.ginassugarshack.ca/,NaN,22.0,5.0
4,Ethos Rejuvenation Centre,Skin care clinic,"Skin care clinic, Day spa, Facial spa, Laser h...",London,ON,5196018881,info@ethosskincare.com,http://www.ethosskincare.com/,Experience the best of medical rejuvenation at...,91.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...
1375,York Medical Group Inc,Skin care clinic,Skin care clinic,Richmond Hill,ON,6478095365,frontdesk@ymgclinic.com,https://ymgclinic.com/,NaN,19.0,5.0
1376,Glow & Go Beauty Medical Spa,Skin care clinic,"Skin care clinic, Cosmetics store, Facial spa,...",Windsor,ON,5198170700,sandra.glowngobeauty@gmail.com,http://www.glowngobeauty.ca/,Glow & Go Beauty is certified in advanced cosm...,10.0,5.0
1377,Essence Beauty By Joelle,Skin care clinic,Skin care clinic,LaSalle,ON,5199916848,None,None,NaN,2.0,5.0
1378,Klodas Medical Aesthetics,Skin care clinic,"Skin care clinic, Facial spa, Medical spa",Toronto,ON,4165353229,None,http://www.klodasmedicalaesthetics.ca/,NaN,5.0,5.0


In [8]:
# Step 4, clean and standardize total_reviews and average_ratings columns to numeric
df["total_reviews"] = pd.to_numeric(df["total_reviews"], errors="coerce")
df["average_rating"] = pd.to_numeric(df["average_rating"], errors="coerce")

df

,clinic_name,clinic_main_type,clinic_sub_type,city,province,phone,email,website_url,website_desc,total_reviews,average_rating
0,DDG Med Spa (Drop Dead Gorgeous),Skin care clinic,"Skin care clinic, Medical spa",Hamilton,ON,9053794344,info@ddgmedspa.com,http://www.ddgmedspa.com/,NaN,203.0,4.7
1,SKYN Med Spa,Skin care clinic,"Skin care clinic, Cosmetic surgeon, Medical spa",Hamilton,ON,9053877596,None,http://www.skynmedspa.ca/,NaN,161.0,4.9
2,Skin Vitality Medical Clinic Hamilton,Skin care clinic,"Skin care clinic, Beauty product supplier, Fac...",Hamilton,ON,3657950096,None,https://www.skinvitality.ca/hamilton/?src=7014...,Huge Savings on CoolSculpting! - Top CoolSculp...,457.0,4.8
3,Gina's Sugar Shack - Organic Skin Care & Hair ...,Skin care clinic,"Skin care clinic, Boutique, Hair removal service",Manotick,ON,6138824435,None,http://www.ginassugarshack.ca/,NaN,22.0,5.0
4,Ethos Rejuvenation Centre,Skin care clinic,"Skin care clinic, Day spa, Facial spa, Laser h...",London,ON,5196018881,info@ethosskincare.com,http://www.ethosskincare.com/,Experience the best of medical rejuvenation at...,91.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...
1375,York Medical Group Inc,Skin care clinic,Skin care clinic,Richmond Hill,ON,6478095365,frontdesk@ymgclinic.com,https://ymgclinic.com/,NaN,19.0,5.0
1376,Glow & Go Beauty Medical Spa,Skin care clinic,"Skin care clinic, Cosmetics store, Facial spa,...",Windsor,ON,5198170700,sandra.glowngobeauty@gmail.com,http://www.glowngobeauty.ca/,Glow & Go Beauty is certified in advanced cosm...,10.0,5.0
1377,Essence Beauty By Joelle,Skin care clinic,Skin care clinic,LaSalle,ON,5199916848,None,None,NaN,2.0,5.0
1378,Klodas Medical Aesthetics,Skin care clinic,"Skin care clinic, Facial spa, Medical spa",Toronto,ON,4165353229,None,http://www.klodasmedicalaesthetics.ca/,NaN,5.0,5.0


In [9]:
# Step 5, perform deduplication

# Step 5.1, deduplicate records with matching clinic_names and city
df = df.drop_duplicates(subset=["clinic_name", "city"], keep='first')
 
# Step 5.2, deduplicate records with matching phone numbers
df = df[df['phone'].isna() | ~df.duplicated(subset=['phone'], keep='first')]

# Step 5.3, deduplicate records with matching emails
df = df[df['email'].isna() | ~df.duplicated(subset=['email'], keep='first')]

df

,clinic_name,clinic_main_type,clinic_sub_type,city,province,phone,email,website_url,website_desc,total_reviews,average_rating
0,DDG Med Spa (Drop Dead Gorgeous),Skin care clinic,"Skin care clinic, Medical spa",Hamilton,ON,9053794344,info@ddgmedspa.com,http://www.ddgmedspa.com/,NaN,203.0,4.7
1,SKYN Med Spa,Skin care clinic,"Skin care clinic, Cosmetic surgeon, Medical spa",Hamilton,ON,9053877596,None,http://www.skynmedspa.ca/,NaN,161.0,4.9
2,Skin Vitality Medical Clinic Hamilton,Skin care clinic,"Skin care clinic, Beauty product supplier, Fac...",Hamilton,ON,3657950096,None,https://www.skinvitality.ca/hamilton/?src=7014...,Huge Savings on CoolSculpting! - Top CoolSculp...,457.0,4.8
3,Gina's Sugar Shack - Organic Skin Care & Hair ...,Skin care clinic,"Skin care clinic, Boutique, Hair removal service",Manotick,ON,6138824435,None,http://www.ginassugarshack.ca/,NaN,22.0,5.0
4,Ethos Rejuvenation Centre,Skin care clinic,"Skin care clinic, Day spa, Facial spa, Laser h...",London,ON,5196018881,info@ethosskincare.com,http://www.ethosskincare.com/,Experience the best of medical rejuvenation at...,91.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...
1374,DeLorenzi Clinic,Skin care clinic,"Skin care clinic, Cosmetics store, Doctor, Fac...",Kitchener,ON,5195702811,None,None,NaN,14.0,3.9
1375,York Medical Group Inc,Skin care clinic,Skin care clinic,Richmond Hill,ON,6478095365,frontdesk@ymgclinic.com,https://ymgclinic.com/,NaN,19.0,5.0
1376,Glow & Go Beauty Medical Spa,Skin care clinic,"Skin care clinic, Cosmetics store, Facial spa,...",Windsor,ON,5198170700,sandra.glowngobeauty@gmail.com,http://www.glowngobeauty.ca/,Glow & Go Beauty is certified in advanced cosm...,10.0,5.0
1378,Klodas Medical Aesthetics,Skin care clinic,"Skin care clinic, Facial spa, Medical spa",Toronto,ON,4165353229,None,http://www.klodasmedicalaesthetics.ca/,NaN,5.0,5.0


In [11]:
# Step 6, remove records that are missing essential fields

# Step 6.1, drop records with missing clinic_name
df = df.dropna(subset=["clinic_name"])

# Step 6.2, drop records with missing email
df = df.dropna(subset=["email"])

# Step 6.3, standardize null columns to None
df = df.where(pd.notnull(df), None)

df

,clinic_name,clinic_main_type,clinic_sub_type,city,province,phone,email,website_url,website_desc,total_reviews,average_rating
0,DDG Med Spa (Drop Dead Gorgeous),Skin care clinic,"Skin care clinic, Medical spa",Hamilton,ON,9053794344,info@ddgmedspa.com,http://www.ddgmedspa.com/,None,203.0,4.7
4,Ethos Rejuvenation Centre,Skin care clinic,"Skin care clinic, Day spa, Facial spa, Laser h...",London,ON,5196018881,info@ethosskincare.com,http://www.ethosskincare.com/,Experience the best of medical rejuvenation at...,91.0,4.4
5,Spectrum Of Beauty Permanent Makeup & Skin Car...,Skin care clinic,Skin care clinic,Richmond Hill,ON,6478685828,info@spectrumofbeauty.ca,http://spectrumcollege.ca/,None,225.0,5.0
6,Toronto Beauty Clinic - North York - Sheppard ...,Skin care clinic,"Skin care clinic, Cosmetics industry, Facial s...",North York,ON,4163980111,hello@torontobeautyclinic.ca,https://www.torontobeautyclinic.ca/,Toronto's best PRP hair care and skin care cli...,195.0,4.6
8,Healthy Image,Skin care clinic,"Skin care clinic, Dermatologist, Medical spa",Windsor,ON,5199719542,contact@healthyimage.ca,http://www.healthyimage.ca/,"Healthy Image Center, led by Dr. Jerry Tan, fo...",38.0,4.5
...,...,...,...,...,...,...,...,...,...,...,...
1353,BE YOU,Skin care clinic,Skin care clinic,Concord,ON,None,info@beyoucanada.ca,http://beyoucanada.ca/,BE YOU luxury skin care products and cosmetics...,1.0,1.0
1360,Total Beauty Care by Silde,Skin care clinic,"Skin care clinic, Electrolysis hair removal se...",Ottawa,ON,6132242666,totalbeautycarebysilde@gmail.com,https://www.silde.ca/,Total Beauty Care by Silde Offering personaliz...,47.0,4.9
1366,LM Injectables,Skin care clinic,"Skin care clinic, Beautician, Health and beaut...",Toronto,ON,4169894073,info@lminjectables.com,http://www.lminjectables.com/,LM Injectables is a Toronto Medical Aesthetics...,32.0,5.0
1375,York Medical Group Inc,Skin care clinic,Skin care clinic,Richmond Hill,ON,6478095365,frontdesk@ymgclinic.com,https://ymgclinic.com/,None,19.0,5.0
